### Create a YouTube Video Summarizer Using Whisper and LangChain 

In [ ]:
import yt_dlp

def download_mp4_from_youtube(url):
    # Set the options for the download
    filename = 'lecuninterview.mp4'
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'outtmpl': filename,
        'quiet': True,
    }

    # Download the video file
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=True)

url = "https://www.youtube.com/watch?v=mBjPyte2ZZo"
download_mp4_from_youtube(url)

In [1]:
import whisper

model = whisper.load_model("tiny")

d:\Bilal\ActiveLoop\env\lib\site-packages\whisper\timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


In [ ]:
result = model.transcribe("lecuninterview.mp4")
print(result['text'])

In [3]:
with open ('text.txt', 'w') as file:  
    file.write(result['text'])

In [4]:
from langchain import OpenAI, LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(model_name="text-davinci-003", temperature=0)

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
)

In [6]:
from langchain.docstore.document import Document

with open('text.txt') as f:
    text = f.read()

texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in texts[:4]]

In [7]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

chain = load_summarize_chain(llm, chain_type="map_reduce")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

 Jan Lekun, a deep learning developer and proponent of self-supervised learning, discusses his joint
embedding predictive architecture and theory of consciousness. He explains how large language models
are missing something and how his architecture may be a step towards filling that gap. He also talks
about supervised learning, large language models, and generative models used to predict missing
words in text. His research could potentially lead to AI systems exhibiting features of
consciousness.


In [8]:
print( chain.llm_chain.prompt.template )

Write a concise summary of the following:


"{text}"


CONCISE SUMMARY:


In [9]:
prompt_template = """Write a concise bullet point summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["text"])

In [10]:
chain = load_summarize_chain(llm, 
                             chain_type="stuff", 
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary, 
                             width=1000,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)


- Jan Lekun is a seminal figure in deep learning development and a long time proponent of self supervised learning
- He spoke about what's missing in large language models and his new joint embedding predictive architecture
- He also discussed his theory of consciousness and the potential for AI systems to exhibit features of consciousness
- Self supervised learning has revolutionized natural language processing and is used in content moderation systems
- Large language models are generative models which are difficult to represent uncertain predictions


In [11]:
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

  Craig Smith interviews Jan Lekun, a deep learning developer and proponent of self-supervised
learning, about his new joint embedding predictive architecture and his theory of consciousness. Jan
discusses how large language models are missing something and how his architecture may be a step
towards filling that gap, as well as the potential for AI systems to exhibit features of
consciousness. He explains how self-supervised learning has revolutionized natural language
processing through the use of transformer architectures for pre-training, such as replacing words
with black markers and training a neural net to predict the missing words. This technique has been
used in practical applications such as content moderation systems on Facebook, Google, YouTube, and
more. Jan also explains how this technique can be used to represent uncertainty in predictions, such
as in video frames, and how it can be applied to other areas such as video.
